# Exercício 3 - grupo 6 - Ana Margarida Campos (A85166) , Nuno Pereira (PG42846)

Neste primeiro exercício foi proposta a implementação de duas versões, IND-CPA e IND-CCA, do protótipo **BIKE** que foi candidato ao concurso *NIST-PQC*, neste caso da terceira ronda.
<br/>
De seguida, é apresentada apenas uma das versões, sendo esta correspondente ao IND-CPA. Os resultados da resolução deste exercício acompanhado de uma explicação detalhada de cada função implementada. Este, foi desenvolvido tendo por base o documento *BIKE_Spec.2020.10.22.1.pdf*. 

Para a resolução deste exercício recorremos a várias funções presentes no notebook *BitFlip-BIKE.ipynb* disponibilizado pelo docente.
<br/>
Para além destas, foi necessário definir outras funções auxiliares sendo estas:

- ***H***: função que recorre ao algoritmo de cifragem *CTR*;

- ***L*** e ***K_sha***: funções de *hash* que utilizam *SHA384*;

- ***bxor***: calcula a operação *XOR* entre duas sequências de *bytes*.

Como funções principais foram implementadas as seguintes funções com os seguintes propósitos:

- ***KeyGen***: gera as chaves pública e privada que vão ser necessárias no encapsulmanto e desincapsulamento. A chave pública corresponde a variàvel *h* e a chave privada é divida por duas partes sendo a primeira constituída por *h0*, *h1* e a segunda constituída por *sigma*;

- ***encaps***: função responsável pelo encapsulamento;

- ***decaps***: função reponsável pelo desencapsulamento.

In [1]:
# imports necessários para a implementação
import random as rn
from cryptography.hazmat.primitives import hashes
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

In [2]:
# constantes BIKE
r = 257
n = 2*r
t = 16
l = 256

K = GF(2)
um = K(1)
zero = K(0)

Vn  = VectorSpace(K,n)
Vr  = VectorSpace(K,r)
Vq  = VectorSpace(QQ,r)

Mr  = MatrixSpace(K,n,r)

R = PolynomialRing(K,name='w')
w = R.gen()
Rr = QuotientRing(R,R.ideal(w^r+1))

def mask(u,v):
    return u.pairwise_product(v)

def hamm(u):
    return sum([1 if a == um else 0 for a in u])

# gera uma palavra de bit aleatória com tamanho recebido como parâmetro
def geraBits(l):
    bits = ""
    for i in range(l):
        bits = bits + str(rn.randint(0,1))
    return bits

# função que recorre ao algoritmo de cifragem CTR
def H(key, m):
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CTR(iv))
    encryptor = cipher.encryptor()
    ct = encryptor.update(m) + encryptor.finalize()
    return ct

# função de hash que utiliza SHA384
def L(e0,e1):
    digest = hashes.Hash(hashes.SHA384())
    digest.update(e0)
    digest.update(e1)
    r = digest.finalize()
    return r

# função de hash que utiliza SHA384
def K_sha(m,C):
    c1, c2 = C
    digest = hashes.Hash(hashes.SHA384())
    digest.update(m)
    digest.update(c1)
    digest.update(c2)
    r = digest.finalize()
    return r

def rot(h):
    v = Vr() ; v[0] = h[-1]
    for i in range(r-1):
        v[i+1] = h[i]
    return v

def Rot(h):
    M = Matrix(K,r,r) ;M[0] = expand(h)
    for i in range(1,r):
        M[i] = rot(M[i-1])
    return M

def expand(f):
    fl = f.list(); ex = r - len(fl)
    return Vr(fl + [zero]*ex)

def expand2(code):
    (f0,f1) = code
    f = expand(f0).list() + expand(f1).list()
    return Vn(f)

def unexpand2(vec):
    u = vec.list()
    return (Rr(u[:r]),Rr(u[r:]))


# operação XOR entre duas sequências de bytes
def bxor(a, b):
    return bytes([ x^^y for (x,y) in zip(a, b)])

# bit flip
def BF(H,code,synd,cnt_iter=r, errs=0):
    mycode = code
    mysynd = synd
    
    while cnt_iter > 0 and hamm(mysynd) > errs:
        cnt_iter   = cnt_iter - 1
        
        unsats     = [hamm(mask(mysynd,H[i])) for i in range(n)]
        max_unsats = max(unsats)
        
        for i in range(n):
            if unsats[i] == max_unsats:
                mycode[i]  += um               ## bit-flip
                mysynd     += H[i]
    
    return mycode     


def sparse_pol(sparse=3):
    coeffs = [1]*sparse + [0]*(r-2-sparse)
    rn.shuffle(coeffs)
    return Rr([1]+coeffs+[1])

# geração das chaves pública e privada
def keyGen():
    while True:
        h0 = sparse_pol(); h1 = sparse_pol()
        if h0 != h1 and h0.is_unit() and h1.is_unit(): # primeira parte da chave privada
            break
    h = h0 * h1.inverse_of_unit() # chave pública
    sigma = geraBits(l) # segunda parte da chave privada
    return h0,h1,sigma,h

# encapsulamento
def encaps(h, key):
    m = geraBits(32) 
    e = H(key, m.encode())
    e0 = e[:16]
    e1 = e[-16:]
    c = (e0 + e1, bxor(m.encode(),L(e0,e1)) )
    k = K_sha(m.encode(),c)
    return k,c

# desencapsulamento
def decaps(h0, h1, sigma, C, key):
    c0, c1 = C
    cr = (Rr(list(c0)), Rr(list(c1)))
    code = expand2(cr)
    H_matrix = block_matrix(2,1,[Rot(h0),Rot(h1)])
    synd = code * H_matrix
    e = BF(H_matrix,code,synd)
    (e0,e1) = unexpand2(e)  
    m = bxor(c1, L(bytes(e0), bytes(e1)))
    if bytes(e) == H(key,m):
        k = K_sha(m,C)
    else:
        k = K_sha(sigma.encode(), C)
    return k
    


De seguida são apresentados os resultados obtidos com recurso às funções anteriores.

In [4]:
key = os.urandom(32)
h0, h1, sigma, h = keyGen()
print("Chave pública: ", h, "\n")
print("Chave privada: h0=", h0, " h1=", h1, " sigma=", sigma, "\n")
k, C = encaps(h, key)
print("Resultado do encapsulamento: k=",k," C=",C, "\n")
desencapsulation = decaps(h0,h1,sigma,C, key)
print("Resultado do desencapsulamento:", desencapsulation)

Chave pública:  wbar^256 + wbar^255 + wbar^254 + wbar^253 + wbar^252 + wbar^251 + wbar^250 + wbar^249 + wbar^248 + wbar^246 + wbar^244 + wbar^243 + wbar^242 + wbar^239 + wbar^237 + wbar^234 + wbar^231 + wbar^229 + wbar^228 + wbar^224 + wbar^223 + wbar^221 + wbar^220 + wbar^218 + wbar^217 + wbar^215 + wbar^214 + wbar^212 + wbar^210 + wbar^209 + wbar^207 + wbar^205 + wbar^203 + wbar^202 + wbar^200 + wbar^198 + wbar^196 + wbar^195 + wbar^192 + wbar^185 + wbar^184 + wbar^183 + wbar^181 + wbar^180 + wbar^179 + wbar^178 + wbar^177 + wbar^176 + wbar^174 + wbar^169 + wbar^168 + wbar^166 + wbar^162 + wbar^158 + wbar^157 + wbar^154 + wbar^151 + wbar^150 + wbar^149 + wbar^148 + wbar^146 + wbar^145 + wbar^144 + wbar^143 + wbar^142 + wbar^139 + wbar^138 + wbar^137 + wbar^136 + wbar^134 + wbar^133 + wbar^130 + wbar^127 + wbar^124 + wbar^123 + wbar^122 + wbar^118 + wbar^115 + wbar^114 + wbar^112 + wbar^111 + wbar^108 + wbar^106 + wbar^104 + wbar^103 + wbar^102 + wbar^97 + wbar^95 + wbar^92 + wbar^91 